In [2]:
import pymysql
import pandas as pd

In [5]:
countries_forms = pd.read_csv('countries_forms.csv')

In [61]:
countries_forms.columns

Index([u'Country', u'Year', u'Electricity source', u'Biomass', u'Coal',
       u'Natural Gas', u'Geothermal', u'Hydro', u'Nuclear', u'Oil', u'Solar',
       u'Wind'],
      dtype='object')

In [60]:
countries_forms.rename(index=str, columns={'Biomass and Waste': 'Biomass', 'Gas': 'Natural Gas', 'Hydroelectric': 'Hydro', 'Solar Tide Wave': 'Solar'}, inplace=True)

In [62]:
energy_forms = ['Natural Gas', 'Biomass', 'Coal', 'Geothermal', 'Hydro', 'Nuclear', 'Oil', 'Solar', 'Wind']

In [57]:
a = 0 if countries_forms.iloc[0]['Hydroelectric'] == '-' else float(countries_forms.iloc[0]['Hydroelectric'])
a

0.579999983

In [23]:
connection = pymysql.connect(host='proj1.ci4g2wbj7lrc.us-west-2.rds.amazonaws.com', user='rip_us', password='abdu9000', db='proj', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

In [28]:
res = None
with connection.cursor() as cursor:
    sql = 'Select * from Form'
    cursor.execute(sql)
    res = cursor.fetchall()

In [35]:
with connection.cursor() as cursor:
    sql = 'CREATE TABLE Country (Name varchar(255), Description varchar(1000), LandArea real, Primary Key (Name))'
    cursor.execute(sql)

In [38]:
with connection.cursor() as cursor:
    for c in countries_forms['Country']:
        sql = 'INSERT INTO Country(Name, Description, LandArea) VALUES (\"' + c + '\", NULL, NULL)'
        cursor.execute(sql)

/Users/sanjays1/anaconda2/lib/python2.7/site-packages/pymysql/cursors.py:323: Warning: (1300, u"Invalid utf8mb4 character string: '997465'")
  self._do_get_result()
/Users/sanjays1/anaconda2/lib/python2.7/site-packages/pymysql/cursors.py:323: Warning: (1366, u"Incorrect string value: '\\x99te d'...' for column 'Name' at row 1")
  self._do_get_result()
/Users/sanjays1/anaconda2/lib/python2.7/site-packages/pymysql/cursors.py:323: Warning: (1300, u"Invalid utf8mb4 character string: '8E756E'")
  self._do_get_result()
/Users/sanjays1/anaconda2/lib/python2.7/site-packages/pymysql/cursors.py:323: Warning: (1366, u"Incorrect string value: '\\x8Eunion' for column 'Name' at row 1")
  self._do_get_result()


In [43]:
with connection.cursor() as cursor:
    sql = 'CREATE TABLE Uses (cname varchar(255), fname varchar(255), percent_usage real, PRIMARY KEY (cname,fname), FOREIGN KEY (cname) REFERENCES Country(Name), FOREIGN KEY (fname) REFERENCES Form(Name))'
    cursor.execute(sql)

In [65]:
with connection.cursor() as cursor:
    for i in range(len(countries_forms)):
        usages = {f: 0.0 for f in energy_forms}
        total = 0.0
        for col in energy_forms:
            entry = countries_forms.iloc[i][col]
            usages[col] = 0. if entry == '-' else float(entry)
            total += usages[col]
        if total > 0:
            for f in energy_forms:
                usages[f] /= total
                if usages[f] > 0:
                    sql = 'INSERT INTO Uses(cname, fname, percent_usage) VALUES (\"' + countries_forms.iloc[i]['Country'] + '\", \"' + f + '\", ' + str(usages[f]) + ')'
                    cursor.execute(sql)

/Users/sanjays1/anaconda2/lib/python2.7/site-packages/pymysql/cursors.py:323: Warning: (1366, u"Incorrect string value: '\\x99te d'...' for column 'cname' at row 1")
  self._do_get_result()
/Users/sanjays1/anaconda2/lib/python2.7/site-packages/pymysql/cursors.py:323: Warning: (1366, u"Incorrect string value: '\\x8Eunion' for column 'cname' at row 1")
  self._do_get_result()


In [66]:
connection.commit()